In [2]:
!pip3 install rpyc

    100% |################################| 71kB 3.1MB/s ta 0:00:011    89% |############################    | 61kB 9.4MB/s eta 0:00:01
    100% |################################| 133kB 2.8MB/s eta 0:00:01


In [1]:
!pip3 show rpyc

Name: rpyc
Version: 5.0.1
Summary: Remote Python Call (RPyC), a transparent and symmetric RPC library
Home-page: http://rpyc.readthedocs.org
Author: Tomer Filiba
Author-email: tomerfiliba@gmail.com
License: MIT
Location: /usr/local/lib/python3.6/dist-packages
Requires: plumbum


In [1]:
import rpyc
from rpyc.utils.server import ThreadedServer
from jetbot import Robot, Camera
import logging
import sys
import cv2
import numpy as np
import base64
import torch

In [2]:


# Set up logging
logging.basicConfig(level=logging.DEBUG)
logger = logging.getLogger('JetBotServer')

class JetBotService(rpyc.Service):
    def __init__(self):
        super().__init__()
        self.robot = None
        self.camera = None
        logger.info("JetBot service initialized")
    
    def on_connect(self, conn):
        logger.info("Client connected")
    
    def on_disconnect(self, conn):
        logger.info("Client disconnected")
        if self.camera is not None:
                self.camera.stop()
                self.camera = None
                logger.info("Camera stopped")

        if self.robot:
            # Safety stop
            try:
                self.robot.stop()
            except:
                pass
    
    def exposed_set_motors(self, left_speed, right_speed):
        try:
            logger.debug(f"Received motor command: left={left_speed}, right={right_speed}")
            
            if self.robot is None:
                logger.debug("Initializing robot")
                self.robot = Robot()
            
            # Convert to float and set motors
            self.robot.left_motor.value = float(left_speed)
            self.robot.right_motor.value = float(right_speed)
            
            logger.debug("Motors set successfully")
            return True  # Acknowledge success
            
        except Exception as e:
            logger.error(f"Error setting motors: {str(e)}")
            raise
            
    def exposed_get_camera_frame(self):
        try:
            if self.camera is None:
                self.camera = Camera.instance(width=224, height=224)
                logger.info("Camera initialized")
            
            # Get frame from camera
            frame = self.camera.value
            #return torch.tensor(frame)
            # Convert to jpg for efficient transfer
            _, buffer = cv2.imencode('.jpg', frame)
            
            # Convert to base64 string for transfer
            jpg_as_text = base64.b64encode(buffer).decode('utf-8')
            
            return jpg_as_text
            
        except Exception as e:
            logger.error(f"Error getting camera frame: {str(e)}")
            return None

In [ ]:
logger.info("Starting JetBot server on port 18861...")
server = ThreadedServer(
    JetBotService(),
    port=18861,
    protocol_config={
        'allow_all_attrs': True,
        'sync_request_timeout': 30,
        'allow_pickle': True,
    }
)
server.start()

INFO:JetBotServer:Starting JetBot server on port 18861...
INFO:JetBotServer:JetBot service initialized
INFO:JETBOT/18861:server started on [0.0.0.0]:18861
INFO:JETBOT/18861:accepted ('192.168.68.57', 64624) with fd 67
INFO:JETBOT/18861:welcome ('192.168.68.57', 64624)
INFO:JetBotServer:Client connected
DEBUG:JetBotServer:Received motor command: left=0.0, right=0.0
DEBUG:JetBotServer:Initializing robot
DEBUG:Adafruit_MotorHAT.Adafruit_PWM_Servo_Driver:Reseting PCA9685 MODE1 (without SLEEP) and MODE2
DEBUG:Adafruit_I2C.Device.Bus.1.Address.0X60:Wrote 0x00 to register 0xFA
DEBUG:Adafruit_I2C.Device.Bus.1.Address.0X60:Wrote 0x00 to register 0xFB
DEBUG:Adafruit_I2C.Device.Bus.1.Address.0X60:Wrote 0x00 to register 0xFC
DEBUG:Adafruit_I2C.Device.Bus.1.Address.0X60:Wrote 0x00 to register 0xFD
DEBUG:Adafruit_I2C.Device.Bus.1.Address.0X60:Wrote 0x04 to register 0x01
DEBUG:Adafruit_I2C.Device.Bus.1.Address.0X60:Wrote 0x01 to register 0x00
DEBUG:Adafruit_I2C.Device.Bus.1.Address.0X60:Read 0x01 fro

In [2]:
# server.py (save this on your JetBot)
import rpyc
from rpyc.utils.server import ThreadedServer

class MyService(rpyc.Service):
    def on_connect(self, conn):
        print("Client connected!")

    def on_disconnect(self, conn):
        print("Client disconnected!")

    def exposed_hello(self, name):
        print(f"Hello, {name}!")
        return f"Greetings, {name} from the JetBot!"

server = ThreadedServer(MyService, port=18861)
server.start()

Client connected!
Hello, World!
Client disconnected!


keyboard interrupt!
